In [46]:
from elasticsearch import Elasticsearch
import re
import os
import pandas as pd
import string
from elasticsearch_dsl import Search
import json
import math

In [47]:
es = Elasticsearch("http://localhost:9200", timeout=60)

In [48]:
os.chdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA")
files = os.listdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection")
files.remove("readme")

In [49]:
# function (also used in parser/indexer) to get list of docnums
def get_docnums():
    test_keys = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = "latin-1")
        text = current.read()
        keys = re.findall("<DOCNO>(.*)</DOCNO>", text)
        for j in keys:
            test_keys.append(j)
    return test_keys

In [50]:
# helper function for get_queries() to return the number of each query
def get_queries_nums(queries_list):
    queries_nums = []
    for line in queries_list:
        num = line.split()[0]
        queries_nums.append(num)
    return queries_nums

In [51]:
# helper function for get_queries() to return necessary text of each query with
def get_queries_text(queries_list):
    queries_nums = get_queries_nums(queries_list)
    query_stop_words = ["Document", "will", "must", "discuss", 
    "report", "include", "describe", "identify", "a", "an", "as",
    "and", "the", "to", "or", "either", "of", "by", "in", "with", 
    "about", "some", "any", "its", "even", "other", "which",
    "being", "certain", "has"]
    remove_words = queries_nums + query_stop_words
    queries_text = []
    for i in queries_list:
        x = i.split()
        queries_text.append(" ".join(a if a not in remove_words else '' for a in x))
    return queries_text

In [52]:
# function to get dataframe with query number and query text
def get_queries_df():
    queries_df = pd.DataFrame()
    with open("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/query_desc.51-100.short.txt") as queries_file:
        queries_list = queries_file.read()
        queries_list = re.sub("-", " ", queries_list)
        queries_list = queries_list.translate(str.maketrans('', '', string.punctuation))
        queries_list = queries_list.split("\n")
        queries_nums = get_queries_nums(queries_list)
        queries_text = get_queries_text(queries_list)
        queries_df["QueryNumber"] = queries_nums
        queries_df["QueryText"] = queries_text
    return queries_df

#queries_df = get_queries_df()
#print(queries_df)

In [53]:
# function to get list of queries 
def get_queries_words(queries_df):
    query_words_list = []
    for query in queries_df.iloc[:, 1]:
        query_words_list.append(query)
    return query_words_list

#queries_df = get_queries_df()
#query_words_list = get_queries_words(queries_df)
#print(query_words_list)

In [54]:
# helper function to search term vector api result for query word and return its frequency within a document - how many times it is in document
def get_tf_num(term, results):
    tf_num = 0
    if results.__contains__("TEXT") is True:
        if term in results["TEXT"]["terms"]:
            tf = results["TEXT"]["terms"][term]["term_freq"]
            tf_num += tf
            return tf_num
        else:
            return tf_num
    else:
        return tf_num

In [55]:
# helper function to search term vector api result for query word and return its document frequency - how many documents it is in
def get_df_num(term, results):
    df_num = 0
    if results.__contains__("TEXT") is True:
        if term in results["TEXT"]["terms"]:
            df = results["TEXT"]["terms"][term]["doc_freq"]
            df_num += df
            return df_num
        else:
            return df_num
    else:
        return df_num

In [56]:
# helper function to get term frequency of query word within the query - how many times is it in the query
def get_tq_num(term, query):
    tq_num = query.count(term)
    return tq_num

In [57]:
# helper function to retrieve dictionary of query words tf within a document
def get_docnum_tf_dict(docnum, query):
    q_word_dict = {}
    results = es.termvectors(index="ap89_index4",
                        id=str(docnum),
                        body={
                            "fields": ["TEXT"],
                            "term_statistics": True,
                            "field_statistics": True
                        })["term_vectors"]
    for term in query:
        q_word_dict[term] = [get_tf_num(term, results), get_df_num(term, results), get_tq_num(term, query)]
    
    return q_word_dict

In [58]:
# function to retrieve term frequency values and document frequency values of words in all 25 queries in each of the 84678 documents
# returns dictionary of {docnum : {q_word : [tf, df, tq], q_word : [tf, df, tq]}, docnum : {q_word : [tf, df, tq], q_word : [tf, df, tq]}, etc.}
def create_tf_df_tq_dict():
    queries_df = get_queries_df()
    q_nums_list = queries_df.iloc[:, 0]
    q_words_list = queries_df.iloc[:, 1]
    test_keys = get_docnums()

    tf_df_tq_dict_list = []
    for list in q_words_list[:25]:
        query = list.split()
        tf_dict = {}
        for docnum in test_keys[:84678]:
            tf_dict[docnum] = get_docnum_tf_dict(docnum, query)
        tf_df_tq_dict_list.append(tf_dict)
    
    return tf_df_tq_dict_list

#tf_df_tq_dict_list = create_tf_df_tq_dict()
#print(tf_df_tq_dict_list)

In [66]:
# helper function to get the doc length of a given doc
def get_doc_length(doc):
    doc_length = 0
   
    results = es.termvectors(index="ap89_index4",
                        id=str(doc),
                        body={
                            "fields": ["TEXT"],
                            "term_statistics": True,
                            "field_statistics": True
                        })["term_vectors"]

    if results.__contains__("TEXT") is True:
        for term in results["TEXT"]["terms"]:
            tf_val = 1 * (results["TEXT"]["terms"][term]["term_freq"])
            doc_length += tf_val
        return doc_length
    else:
        return doc_length

In [61]:
# helper function to get dataframe of okapi bm25 scores for 1 query for all documents
def get_okapi_bm25_df(tf_df_tq_dict, count):
 
   doc_length_list = []
   for doc in tf_df_tq_dict.keys():
      doc_length = get_doc_length(doc)
      doc_length_list.append(doc_length)
   
   doc_id_list = []
   for doc in tf_df_tq_dict.keys():
      doc_id_list.append(doc)

   test_keys = get_docnums()
   corpus_size = len(test_keys)
   avg_doc_length = (sum(doc_length_list))/corpus_size

   queries_df = get_queries_df()
   q_n_list = queries_df["QueryNumber"].tolist()

   okapi_bm25_score = []
   k1 = 1.2
   k2 = 100
   b = 0.75
   q_tf_df_tq_list = [i for i in tf_df_tq_dict.values()]
   for index, list in enumerate(q_tf_df_tq_list):
      current_list = []
      okapi_b = 0
      for tf_df_tq in list.values():
         doc_length = doc_length_list[index]

         numer = corpus_size + 0.5
         denom = tf_df_tq[1] + 0.5
         part_1 = math.log(numer / denom)

         numer2 = tf_df_tq[0] + (k1 * tf_df_tq[0])
         denom2 = tf_df_tq[0] + (k1 * ((1 - b) + (b * (doc_length / (avg_doc_length)))))
         part_2 = numer2 / denom2

         numer3 = tf_df_tq[2] + (k2 * tf_df_tq[2])
         denom3 = tf_df_tq[2] + k2
         part_3 = numer3 / denom3

         okapi_b += (part_1 * part_2 * part_3)
         current_list.append(okapi_b)

      okapi_bm25_score.append(sum(current_list))
   
   bm25_df = pd.DataFrame()
   bm25_df = bm25_df.reset_index()
   bm25_df["QUERYNUM"] = [q_n_list[count]] * len(doc_id_list)
   bm25_df["DOCNO"] = doc_id_list
   bm25_df["SCORE"] = okapi_bm25_score
   result_df = bm25_df.sort_values(by="SCORE", ascending=False)
   result_df = result_df.iloc[:1000]

   return result_df

In [69]:
# main function to write retrieval model results to output file
def okapi_bm25():
    tf_df_tq_dict_list = create_tf_df_tq_dict()

    okapi_bm25_results_list = []
    count = 0
    for tf_df_tq_dict in tf_df_tq_dict_list:
        result_df = get_okapi_bm25_df(tf_df_tq_dict, count)
        count += 1
        okapi_bm25_results_list.append(result_df)
    
    for df in okapi_bm25_results_list:
        query_number_result = df["QUERYNUM"].tolist()
        query_result_docnums = df["DOCNO"].tolist()
        query_result_scores = df["SCORE"].tolist()
        with open('/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/Okapi_BM25_results.txt', 'a') as queryResults:
            rank = 1
            j = 0
            while j in range(len(query_result_docnums)):
                queryResults.write('%s Q0 %s %s %s Exp\n' % (query_number_result[j], query_result_docnums[j], rank, query_result_scores[j]))
                rank += 1
                j += 1
        queryResults.close()

okapi_bm25()